In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sympy import false
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.float_format', '{:.8f}'.format)
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
df = pd.read_csv('data/final.csv')
boolean_columns = df.select_dtypes(include='bool').columns
df[boolean_columns] = df[boolean_columns].astype(int)
city_columns = [col for col in df.columns if col.startswith('city_')]
df['city_label'] = df[city_columns].idxmax(axis=1).str.replace('city_', '')
X = df.drop(columns=['price', 'city_label'])  # Features
y = df['price']  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df['city_label'],
                                                    shuffle=True)

minmax_scaler = StandardScaler()
X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (470012, 63)
X_test shape: (117504, 63)
y_train shape: (470012,)
y_test shape: (117504,)


In [3]:
print("X_train type = ", type(X_train))
print("X_test type = ", type(X_test))
print("y_train type = ", type(y_train))
print("y_test type = ", type(y_test))

X_train type =  <class 'numpy.ndarray'>
X_test type =  <class 'numpy.ndarray'>
y_train type =  <class 'pandas.core.series.Series'>
y_test type =  <class 'pandas.core.series.Series'>


In [4]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # If using DataFrames, convert to .values
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_train_tensor = y_train_tensor.view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
y_test_tensor = y_test_tensor.view(-1, 1)

print("X_train_tensor shape:", X_train_tensor.size())
print("X_test_tensor shape:", X_test_tensor.size())
print("y_train_tensor shape:", y_train_tensor.size())
print("y_test_tensor shape:", y_test_tensor.size())

X_train_tensor shape: torch.Size([470012, 63])
X_test_tensor shape: torch.Size([117504, 63])
y_train_tensor shape: torch.Size([470012, 1])
y_test_tensor shape: torch.Size([117504, 1])


In [5]:
import warnings
warnings.filterwarnings('ignore')

dataloader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=250000, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class FFNN2(nn.Module):
    def __init__(self, input_size):
        super(FFNN2, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)  # No activation here for regression (raw output)
        return x

model = FFNN2(input_size=X_train_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.5)
epochs = 500
model = model.to(device)
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for data, label in dataloader:
        data = data.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, label.float())
        loss.backward()
        optimizer.step()
    # Testing phase
torch.save(model, 'model_final.pt')
print("Model saved!")
# X_train_tensor
# X_test_tensor
model = torch.load('model_final.pt')
model.eval()
with torch.no_grad():
    y_train_pred = model(X_train_tensor.to(device)).cpu().numpy()
    y_test_pred = model(X_test_tensor.to(device)).cpu().numpy()
y_train_numpy = y_train_tensor.numpy()
y_test_numpy = y_test_tensor.numpy()
train_rsme = root_mean_squared_error(y_train_numpy, y_train_pred)
test_rsme = root_mean_squared_error(y_test_numpy, y_test_pred)
train_r2 = r2_score(y_train_numpy, y_train_pred)
test_r2 = r2_score(y_test_numpy, y_test_pred)
print("Train RSME: ", train_rsme)
print("Test RSME: ", test_rsme)
print("Train R2: ", train_r2)
print("Test R2: ", test_r2)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [6]:
"""
Best yet
standard scaler
batch_size=250000, shuffle=True
self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)

tanh
lr=0.5
epochs 500

Train RSME:  0.5912521
Train MSE: 0.34957904575441

Test RSME:  0.5941896
Test MSE: 0.35306128074816

Train R2:  0.4251762628555298
Test R2:  0.41217756271362305
"""

'\nBest yet\nstandard scaler\nbatch_size=250000, shuffle=True\nself.fc1 = nn.Linear(input_size, 256)\n        self.fc2 = nn.Linear(256, 128)\n        self.fc3 = nn.Linear(128, 32)\n        self.fc4 = nn.Linear(32, 1)\n\ntanh\nlr=0.5\nepochs 500\n\nTrain RSME:  0.56502557\nTest RSME:  0.5708311\nTrain R2:  0.47504085302352905\nTest R2:  0.457485556602478\n'